In [1]:
import pandas as pd
import numpy as np


In [19]:
# A function with accepts a path to a cross reference in csv format and returns two dataframes, one for analogs, and one for digitals
def split_xref(path):

    # Read csv to dataframe skipping the SIM4ME row
    df = pd.read_csv(path, index_col=None, skiprows=1)

    # Locate the start of the Digital I/O
    first_d = df.loc[df["#TYPE"] == "#TYPE"].index[0]

    # Create separate dataframes for the analogs and digitals
    a_df = df.iloc[0:first_d]

    # Save dataframe
    #a_df.to_csv("xref/analogs.csv")

    d_df = df.iloc[first_d:]

    # grab the first row for the header
    new_header = d_df.iloc[0] 

    # take the data less the header row
    d_df = d_df[1:] 

    # Reset the header
    d_df.columns = new_header

    # Save dataframe
    #d_df.to_csv("xref/digitals.csv")
    
    return (a_df, d_df)

# A function that grabs the SCP signal field and puts it in the MISC 5 column
def get_scp_field(df, io, scp_names):
    
    df = df.replace(np.nan, '', regex=True)
    to_engine = df.columns.get_loc("TO ENGINE")
    from_engine = df.columns.get_loc("FROM ENGINE")
    to_symbol = df.columns.get_loc("TO SYMBOL")
    from_symbol = df.columns.get_loc("FROM SYMBOL")
    equation = df.columns.get_loc("EQUATION")
    
    scp_lines = []
    
    # Iterate over the rows
    for i in range(len(df)):
        
        scp_flag = False
        
        # Detect if the line is for an SCP engine
        for engine in scp_names:
            
            # If the to engine is an SCP, get the concatenation of the to symbol
            if (df.iloc[i, to_engine] == engine):
                
                scp_flag = True
                scp_lines.append(strip_scp(str(df.iloc[i, to_symbol])))
            
            # If the from engine is an SCP, get the concatenation of the from symbol and equation field
            elif (df.iloc[i, from_engine] == engine):

                scp_flag = True
                scp_lines.append(strip_scp(str(df.iloc[i, from_symbol]) + str(df.iloc[i, equation])))
            
        
        if scp_flag==False:
                
            scp_lines.append("")
        
    df["MISC5"] = scp_lines
    
            
    return df

# A function which takes in an SCP string a returns only the compound:block.point
def strip_scp(my_string):
    
    # Find everything left off the semicolon
    scolon = my_string.find(":")

    scolon_left = my_string[0:scolon]

    # Reverse the string to perform find in backward direction
    str_reversed =''.join(reversed(scolon_left))

    # Define special characters to search for
    characters = ['(', ')', " ", '+', "-","*","/","^", "'"]

    # Find the first instance of a special character
    finds = []
    for character in characters:

        finds.append(str_reversed.find(character))

    # Remove -1 from list
    finds = list(filter(lambda a: a != -1, finds))

    # Get everything up to the first special character
    if finds:
        mystr = str_reversed[0:min(finds)]
    else:
        mystr = str_reversed

    # Reverse to return to normal order
    compound =''.join(reversed(mystr))


    # Find everything right of the semicolon
    scolon_right = my_string[scolon:]

    # Find the first instance of a special character
    finds = []
    for character in characters:

        finds.append(scolon_right.find(character))

    # Remove -1 from list
    finds = list(filter(lambda a: a != -1, finds))

    if finds:
        block_param = scolon_right[:min(finds)]
    else:
        block_param = scolon_right

    scp_key = compound + block_param

    return scp_key
                

In [20]:
# Load in existing cross reference and seperate into analog and digital dataframes
(a_old_df, d_old_df) = split_xref("Pre_upgrade_xref.csv")

# Load in SCP-generated cross reference and seperate into analog and digital dataframes
(a_master_df, d_master_df) = split_xref("Upgrade_master_xref.csv")

# Load in report-generated cross reference and seperate into analog and digital dataframes
(a_report_df, d_report_df) = split_xref("upgrade_report_xref.csv")


In [21]:
# Augment analog and digital dataframes with SCP primary key
d_old_df = get_scp_field(d_old_df, False, ["WANSCP2VM", "WANSCP1VM"])
a_old_df = get_scp_field(a_old_df, False, ["WANSCP2VM", "WANSCP1VM"])
d_master_df = get_scp_field(d_master_df, False, ["WANSCP2VM", "WANSCP1VM"])
a_master_df = get_scp_field(a_master_df, False, ["WANSCP2VM", "WANSCP1VM"])
d_report_df = get_scp_field(d_report_df, False, ["SCP"])
a_report_df = get_scp_field(a_report_df, False, ["SCP"])

# merge old data with new data
a_master_df = a_master_df.merge(a_old_df, how='left', on="MISC5", suffixes=('', '_old'))
d_master_df = d_master_df.merge(d_old_df, how='left', on="MISC5", suffixes=('', '_old'))

# merge report data with new data
a_master_df = a_master_df.merge(a_report_df, how='left', on="MISC5", suffixes=('', '_report'))
d_master_df = d_master_df.merge(d_report_df, how='left', on="MISC5", suffixes=('', '_report'))


In [22]:
#------------/ Analogs /-----------------
old_to_engine = a_master_df.columns.get_loc("TO ENGINE_old")
old_from_engine = a_master_df.columns.get_loc("FROM ENGINE_old")
old_to_symbol = a_master_df.columns.get_loc("TO SYMBOL_old")
old_from_symbol = a_master_df.columns.get_loc("FROM SYMBOL_old")
old_equation = a_master_df.columns.get_loc("EQUATION_old")

report_to_engine = a_master_df.columns.get_loc("TO ENGINE_report")
report_from_value = a_master_df.columns.get_loc("FROM VALUE INFO_report")
report_to_value = a_master_df.columns.get_loc("TO VALUE INFO_report")
report_misc1 = a_master_df.columns.get_loc("MISC1_report")
report_misc2 = a_master_df.columns.get_loc("MISC2_report")
report_misc3 = a_master_df.columns.get_loc("MISC3_report")

to_engine = a_master_df.columns.get_loc("TO ENGINE")
from_engine = a_master_df.columns.get_loc("FROM ENGINE")
to_symbol = a_master_df.columns.get_loc("TO SYMBOL")
from_symbol = a_master_df.columns.get_loc("FROM SYMBOL")
equation = a_master_df.columns.get_loc("EQUATION")
from_value = a_master_df.columns.get_loc("FROM VALUE INFO")
to_value = a_master_df.columns.get_loc("TO VALUE INFO")
misc1 = a_master_df.columns.get_loc("MISC1")
misc2 = a_master_df.columns.get_loc("MISC2")
misc3 = a_master_df.columns.get_loc("MISC3")

to_engine_list = []
from_engine_list = []
to_symbol_list = []
from_symbol_list = []
equation_list = []
from_value_list = []
to_value_list = []
misc1_list = []
misc2_list = []
misc3_list = []

for row in range(len(a_master_df)):
    if row % 500 == 0:
        print(row)
    
    # For each row, if not the TO ENGINE_old is not NAN,
    if pd.isnull(a_master_df.iloc[row, old_to_engine]) == False:
       
        # Overwrite Analog data in SCP-generated cross reference with orignal data in the
        # FROM SYMBOL / TO SYMBOl / EQUATION / TO ENGINE /  FROM ENGINE
        to_engine_list.append(a_master_df.iloc[row, old_to_engine])
        from_engine_list.append(a_master_df.iloc[row, old_from_engine])
        to_symbol_list.append(a_master_df.iloc[row, old_to_symbol])
        from_symbol_list.append(a_master_df.iloc[row, old_from_symbol])
        equation_list.append(a_master_df.iloc[row, old_equation])
    
    # If there isn't legacy information available, take the default
    else:
        
        to_engine_list.append(a_master_df.iloc[row, to_engine])
        from_engine_list.append(a_master_df.iloc[row, from_engine])
        to_symbol_list.append(a_master_df.iloc[row, to_symbol])
        from_symbol_list.append(a_master_df.iloc[row, from_symbol])
        equation_list.append(a_master_df.iloc[row, equation])

        
    # For each row, if not the TO ENGINE_report is not NAN,
    if pd.isnull(a_master_df.iloc[row, report_to_engine]) == False:
       
        # Overwrite Analog data in SCP-generated cross reference with orignal data in the
        # FROM VALUE / TO VALUE / MISC1 / MISC2 / MISC3
        from_value_list.append(a_master_df.iloc[row, report_from_value])
        to_value_list.append(a_master_df.iloc[row, report_to_value])
        misc1_list.append(a_master_df.iloc[row, report_misc1])
        misc2_list.append(a_master_df.iloc[row, report_misc2])
        misc3_list.append(a_master_df.iloc[row, report_misc3])
        
    else:
        
        from_value_list.append(a_master_df.iloc[row, from_value])
        to_value_list.append(a_master_df.iloc[row, to_value])
        misc1_list.append(a_master_df.iloc[row, misc1])
        misc2_list.append(a_master_df.iloc[row, misc2])
        misc3_list.append(a_master_df.iloc[row, misc3])
        
a_master_df["TO ENGINE"] = to_engine_list
a_master_df["FROM ENGINE"] = from_engine_list
a_master_df["TO SYMBOL"] = to_symbol_list
a_master_df["FROM SYMBOL"] = from_symbol_list
a_master_df["EQUATION"] = equation_list
a_master_df["FROM VALUE INFO"] = from_value_list
a_master_df["TO VALUE INFO"] = to_value_list
a_master_df["MISC1"] = misc1_list
a_master_df["MISC2"] = misc2_list
a_master_df["MISC3"] = misc3_list


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500


In [23]:
#------------/ Digitals /-----------------
# FROM SYMBOL / TO SYMBOl / EQUATION / TO ENGINE /  FROM ENGINE
# SET TEST / SET THRESH / RESET TEST / RESET THRESH
old_to_engine = d_master_df.columns.get_loc("TO ENGINE_old")
old_from_engine = d_master_df.columns.get_loc("FROM ENGINE_old")
old_to_symbol = d_master_df.columns.get_loc("TO SYMBOL_old")
old_from_symbol = d_master_df.columns.get_loc("FROM SYMBOL_old")
old_equation = d_master_df.columns.get_loc("EQUATION_old")
old_set_test = d_master_df.columns.get_loc("SET TEST_old")
old_set_thresh = d_master_df.columns.get_loc("SET THRESH_old")
old_reset_test = d_master_df.columns.get_loc("RESET TEST_old")
old_reset_thresh = d_master_df.columns.get_loc("RESET THRESH_old")

# FROM VALUE / TO VALUE / TRUE STATE / FALSE STATE / MISC1 / MISC2 / MISC3
report_to_engine = d_master_df.columns.get_loc("TO ENGINE_report")
report_from_value = d_master_df.columns.get_loc("FROM VALUE INFO_report")
report_to_value = d_master_df.columns.get_loc("TO VALUE INFO_report")
report_true_state = d_master_df.columns.get_loc("TRUE STATE_report")
report_false_state = d_master_df.columns.get_loc("FALSE STATE_report")
report_misc1 = d_master_df.columns.get_loc("MISC1_report")
report_misc2 = d_master_df.columns.get_loc("MISC2_report")
report_misc3 = d_master_df.columns.get_loc("MISC3_report")

to_engine = d_master_df.columns.get_loc("TO ENGINE")
from_engine = d_master_df.columns.get_loc("FROM ENGINE")
to_symbol = d_master_df.columns.get_loc("TO SYMBOL")
from_symbol = d_master_df.columns.get_loc("FROM SYMBOL")
equation = d_master_df.columns.get_loc("EQUATION")
from_value = d_master_df.columns.get_loc("FROM VALUE INFO")
to_value = d_master_df.columns.get_loc("TO VALUE INFO")
true_state = d_master_df.columns.get_loc("TRUE STATE")
false_state = d_master_df.columns.get_loc("FALSE STATE")
set_test = d_master_df.columns.get_loc("SET TEST")
set_thresh = d_master_df.columns.get_loc("SET THRESH")
reset_test = d_master_df.columns.get_loc("RESET TEST")
reset_thresh = d_master_df.columns.get_loc("RESET THRESH")
misc1 = d_master_df.columns.get_loc("MISC1")
misc2 = d_master_df.columns.get_loc("MISC2")
misc3 = d_master_df.columns.get_loc("MISC3")

to_engine_list = []
from_engine_list = []
to_symbol_list = []
from_symbol_list = []
equation_list = []
set_test_list = []
set_thresh_list = []
reset_test_list = []
reset_thresh_list = []
from_value_list = []
to_value_list = []
true_state_list = []
false_state_list = []
misc1_list = []
misc2_list = []
misc3_list = []


for row in range(len(d_master_df)):
    if row % 500 == 0:
        print(row)
    
    # For each row, if not the TO ENGINE_old is not NAN, grab te
    if pd.isnull(d_master_df.iloc[row, old_to_engine]) == False:
       
        # Overwrite Analog data in SCP-generated cross reference with orignal data in the
        # FROM SYMBOL / TO SYMBOl / EQUATION / TO ENGINE /  FROM ENGINE
        # SET TEST / SET THRESH / RESET TEST / RESET THRESH
        to_engine_list.append(d_master_df.iloc[row, old_to_engine])
        from_engine_list.append(d_master_df.iloc[row, old_from_engine])
        to_symbol_list.append(d_master_df.iloc[row, old_to_symbol])
        from_symbol_list.append(d_master_df.iloc[row, old_from_symbol])
        equation_list.append(d_master_df.iloc[row, old_equation])
        set_test_list.append(d_master_df.iloc[row, old_set_test])
        set_thresh_list.append(d_master_df.iloc[row, old_set_thresh])
        reset_test_list.append(d_master_df.iloc[row, old_reset_test])
        reset_thresh_list.append(d_master_df.iloc[row, old_reset_thresh])
    
    else:
        
        to_engine_list.append(d_master_df.iloc[row, to_engine])
        from_engine_list.append(d_master_df.iloc[row, from_engine])
        to_symbol_list.append(d_master_df.iloc[row, to_symbol])
        from_symbol_list.append(d_master_df.iloc[row, from_symbol])
        equation_list.append(d_master_df.iloc[row, equation])
        set_test_list.append(d_master_df.iloc[row, set_test])
        set_thresh_list.append(d_master_df.iloc[row, set_thresh])
        reset_test_list.append(d_master_df.iloc[row, reset_test])
        reset_thresh_list.append(d_master_df.iloc[row, reset_thresh])
        
    # For each row, if not the TO ENGINE_report is not NAN,
    if pd.isnull(d_master_df.iloc[row, report_to_engine]) == False:
       
        # Overwrite Analog data in SCP-generated cross reference with orignal data in the
        # FROM VALUE / TO VALUE / TRUE STATE / FALSE STATE / MISC1 / MISC2 / MISC3
        from_value_list.append(d_master_df.iloc[row, report_from_value])
        to_value_list.append(d_master_df.iloc[row, report_to_value])
        true_state_list.append(d_master_df.iloc[row, report_true_state])
        false_state_list.append(d_master_df.iloc[row, report_false_state])
        misc1_list.append(d_master_df.iloc[row, report_misc1])
        misc2_list.append(d_master_df.iloc[row, report_misc2])
        misc3_list.append(d_master_df.iloc[row, report_misc3])
    
    else:
        
        from_value_list.append(d_master_df.iloc[row, from_value])
        to_value_list.append(d_master_df.iloc[row, to_value])
        true_state_list.append(d_master_df.iloc[row, true_state])
        false_state_list.append(d_master_df.iloc[row, false_state])
        misc1_list.append(d_master_df.iloc[row, misc1])
        misc2_list.append(d_master_df.iloc[row, misc2])
        misc3_list.append(d_master_df.iloc[row, misc3])

d_master_df["TO ENGINE"] = to_engine_list
d_master_df["FROM ENGINE"] = from_engine_list
d_master_df["TO SYMBOL"] = to_symbol_list
d_master_df["FROM SYMBOL"] = from_symbol_list
d_master_df["EQUATION"] = equation_list
d_master_df["SET TEST"] = set_test_list
d_master_df["SET THRESH"] = set_thresh_list
d_master_df["RESET TEST"] = reset_test_list
d_master_df["RESET THRESH"] = reset_thresh_list
d_master_df["FROM VALUE INFO"] = from_value_list
d_master_df["TO VALUE INFO"] = to_value_list
d_master_df["TRUE STATE"] = true_state_list
d_master_df["FALSE STATE"] = false_state_list
d_master_df["MISC1"] = misc1_list
d_master_df["MISC2"] = misc2_list
d_master_df["MISC3"] = misc3_list

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500


In [24]:
# Save for later
a_master_df.to_csv("xref/analogs_master.csv", index=False)
d_master_df.to_csv("xref/digitals_master.csv", index=False)

In [29]:
# Create final dataframes without unwanted columns
a_new_df = a_master_df.iloc[:, :24]
d_new_df = d_master_df.iloc[:, :23]

In [38]:
# Move all values from the FROM SYMBOL to the EQUATION field
equation = d_new_df.columns.get_loc("EQUATION")
equation_list = []
from_symbol_list = [] 

for row in range(len(d_new_df)):
    if row % 500 == 0:
        print(row)
    
    # For each row, if not the EQUATION is not blank,
    if d_new_df.iloc[row, equation] == '':
        
        # Set equation to From Symbol
        equation_list.append(d_new_df.iloc[row, from_symbol])
    
    else:
        
        equation_list.append(d_new_df.iloc[row, equation])
    
        # Set From Symbol to blank
     
    from_symbol_list.append(np.nan)

d_new_df["FROM SYMBOL"] = from_symbol_list
d_new_df["EQUATION"] = equation_list


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500


In [33]:
# Move all values from the FROM SYMBOL to the EQUATION field
equation = a_new_df.columns.get_loc("EQUATION")
equation_list = []
from_symbol_list = [] 

for row in range(len(a_new_df)):
    if row % 500 == 0:
        print(row)
    
    # For each row, if not the EQUATION is blank, get the information from the FROM SYMBOL
    if a_new_df.iloc[row, equation] == '':
        
        # Set equation to From Symbol
        equation_list.append(a_new_df.iloc[row, from_symbol])
    
    else:
        
        equation_list.append(a_new_df.iloc[row, equation])
    
        # Set From Symbol to blank
     
    from_symbol_list.append(np.nan)

a_new_df["FROM SYMBOL"] = from_symbol_list
a_new_df["EQUATION"] = equation_list


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500


In [35]:
print(equation_list[0])
print(a_new_df.iloc[0, equation])
print(a_new_df.iloc[0, from_symbol])

ZERO
ZERO
nan


In [41]:
# Save for later
a_new_df.to_csv("xref/analogs.csv", index=False)
d_new_df.to_csv("xref/digitals.csv", index=False)

In [39]:
d_new_df.head()

,#TYPE,FROM ENGINE,FROM SYMBOL,FROM VALUE INFO,FROM VALUE UOM,TO ENGINE,TO SYMBOL,TO VALUE INFO,PERIOD,EQUATION,...,RESET TEST,RESET THRESH,TRUE STATE,FALSE STATE,MISC1,MISC2,MISC3,MISC4,MISC5,COMMENTS
0,D,DynsimPEngine,NaN,refs,,WANSCP1VM,1AIR5:CC227.FBCIN,1D1403:7,,DIGFALSE,...,,,LOW,NORMAL,CIN : 1CP161 : UNIT 1 CC227,IVO = 0,,,1AIR5:CC227.FBCIN,
1,D,DynsimPEngine,NaN,refs,,WANSCP1VM,1AIR5:CC228.FBCIN,1D1403:6,,DIGFALSE,...,,,ALARM,NORMAL,CIN : 1CP161 : UNIT 1 CC228,IVO = 0,,,1AIR5:CC228.FBCIN,
2,D,DynsimPEngine,NaN,refs,,WANSCP1VM,1AIR5:CC652.FBCIN,1D1318:2,,DIGFALSE,...,,,HIGH,NORMAL,CIN : 1CP161 : UNIT 1 CC652,IVO = 0,,,1AIR5:CC652.FBCIN,
3,D,DynsimPEngine,NaN,refs,,WANSCP1VM,1AIR5:CC653.FBCIN,1D1321:14,,DIGFALSE,...,,,HIGH,NORMAL,CIN : 1CP161 : UNIT 1 CC653,IVO = 0,,,1AIR5:CC653.FBCIN,
4,D,DynsimPEngine,NaN,refs,,WANSCP1VM,1AIR5:CC654.FBCIN,1D1322:15,,DIGFALSE,...,,,HIGH,NORMAL,CIN : 1CP161 : UNIT 1 CC654,IVO = 0,,,1AIR5:CC654.FBCIN,
